# Using RAG with Word files



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### imports

In [ ]:
!pip install llama-index
!pip install llama-index-embeddings-huggingface
!pip install peft
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

### Define Settings

In [ ]:
# import any embedding model on HF hub (https://huggingface.co/spaces/mteb/leaderboard)
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
# Settings.embed_model = HuggingFaceEmbedding(model_name="thenlper/gte-large") # alternative model

Settings.llm = None             #  in this case, the system will only use an embedding model and not a full-fledged LLM
Settings.chunk_size = 100       #  input text is being divided into chunks of 256 tokens/characters (depending on the implementation)
Settings.chunk_overlap = 25     # there is an overlap of 25 tokens/characters between consecutive chunks. Overlap helps maintain context continuity between chunks

### Read and Store Docs into Vector DB

In [ ]:
import os

# check if storage already exists
PERSIST_DIR = "/content/drive/MyDrive/VQA-Final/miscellanous dataset/RAG_Context/Storage"

if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("/content/drive/MyDrive/VQA-Final/miscellanous dataset/RAG_Context/documents").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)


### Set Up Search Function

In [ ]:
# set number of docs to retreive
top_k = 1

# configure retriever
retriever = VectorIndexRetriever(
    index=index,                                # retrives the similar documents from created index
    similarity_top_k=top_k,
)

In [ ]:
# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

### Retrieve Relevant Docs

In [ ]:
# query documents
query = "Where is Taleju Temple?"
response = query_engine.query(query)
response

In [ ]:
# Reformat response to form a paragraph
context = "Context:\n"
for i in range(top_k):
    # Append each text from the source nodes, stripping excess newlines and spaces
    context += response.source_nodes[i].text.replace("\n", " ").strip() + " "

# Remove extra spaces at the start/end and print the final formatted context
context = context.strip()

# Print the reformatted response as a paragraph
print(context)


# Retrive the context


In [1]:
!pip install wandb
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=d22ab900980aac8cb6df3683332dc9cd5a052f64622028ffe2b1e2a435b2b5cd
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import wandb
from rouge_score import rouge_scorer
# from dotenv import load_dotenv
import os
from tabulate import tabulate
from nltk.translate.bleu_score import corpus_bleu
import sympy as sp
# load_dotenv()

In [4]:
df = pd.read_csv('/kaggle/input/test-rag/updated_dataset.csv')

# If 'Unnamed: 0' column still exists, you can drop it
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

In [ ]:
values = df['object'].value_counts()
values

In [ ]:
df = df[~(df['object'] == "what are the customary actions performed at the conclusion of a prayer wheel practice session?\"")]

In [ ]:
values = df['object'].value_counts()
values

In [ ]:
# 80% -> Training Data, 20% -> Testing Data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 90% -> Training Data, 10% -> Validation Data
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

In [ ]:
train_df

In [ ]:
values = train_df['object'].value_counts()
values

In [ ]:
# Placeholder function for retrieving context using RAG
def retrieve_context(question):
    # Here, you would implement your RAG querying logic
    # For example, using the query engine you've set up
    context = ""
    response = query_engine.query(question)  # Replace with actual querying method
    # Check if source_nodes exist in the response and get the number of available nodes
    num_nodes = len(response.source_nodes) if response.source_nodes else 0

    # Iterate only up to the available number of nodes or top_k, whichever is smaller
    for i in range(min(top_k, num_nodes)):
        # Append each text from the source nodes, stripping excess newlines and spaces
        context += response.source_nodes[i].text.replace("\n", " ").strip() + " "

    # Remove extra spaces at the start/end and print the final formatted context
    context = context.strip() # Adjust based on how your response object is structured
    return context

In [ ]:
# Add a new column for context
df['Context'] = ''

# Iterate through each question and retrieve the context
for index, row in df.iterrows():
    question = row['question']
    # Retrieve context for the current question
    context = retrieve_context(question)
    # Update the DataFrame with the retrieved context
    df.at[index, 'Context'] = context

# Save the updated DataFrame to a new CSV file
output_file_path = 'updated_dataset'  # Replace with your desired output file path
# Define an escape character (e.g., backslash)
df.to_csv("/content/drive/MyDrive/VQA-Final/miscellanous dataset/RAG_Context/updated_dataset.csv", index=False, escapechar='\\')

print("Context added and saved to:", output_file_path)

# BART QA with context Training

In [5]:
df = pd.read_csv('/kaggle/input/test-rag/updated_dataset.csv')

# # If 'Unnamed: 0' column still exists, you can drop it
# df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# 80% -> Training Data, 20% -> Testing Data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# 90% -> Training Data, 10% -> Validation Data
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

In [6]:
train_df

,question,object,answer,Context
3791,How many arms can Bhairava be depicted with?,kala bhairav,"Bhairava may be depicted with four, eight, or ...","Often shown with a wrathful expression, bared ..."
6210,How did the brothers celebrate completing the ...,boudhanath,"They stood in front of it and made prayers, ge...",Why? Because it is so powerful that the wishes...
1515,What specific physical characteristics must a ...,taleju bell,"The girl must be in excellent health, never ha...",The king and other religious leaders that migh...
1469,During which event does the Nepalese King typi...,taleju bell,The Nepalese King seeks the blessing of the Ro...,The Kumari is also revered and worshipped by s...
2763,What does the prayer wheel represent in Buddhi...,prayer wheel,The prayer wheel embodies all the actions of t...,"A prayer wheel, or mani wheel, is a cylindrica..."
...,...,...,...,...
6165,What relevance does the Bouddha Stupa legend h...,boudhanath,It is considered important and incredibly insp...,A painting of Samvari is on the rear of the Pu...
310,What does the Nara Yali emblem signify in trad...,yali,The Nara Yali symbolizes the protection of dha...,"Kanjivaram sari, it is intricately woven with ..."
995,Which individual is currently recognized as th...,Taleju Temple,The current Royal Kumari is Trishna Shakya age...,The Kumari is also revered and worshipped by s...
6310,How has the adoption of Mahayana Buddhism evol...,boudhanath,It has spread and been preserved for many year...,So they stayed around him on the mountain to p...


In [7]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

def calculate_max_length(column_name):
    df[column_name] = df[column_name].astype(str)
    return df[column_name].apply(lambda x: len(tokenizer.tokenize(x))).max()

max_length_question = calculate_max_length('question')
max_length_object = calculate_max_length('object')
max_length_context = calculate_max_length('Context')
max_length_answer = calculate_max_length('answer')

print(f"Maximum token length in the 'Input' : {max_length_question + max_length_object + max_length_context}")
print(f"Maximum token length in the 'Output' : {max_length_answer}")

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Maximum token length in the 'Input' : 182
Maximum token length in the 'Output' : 57


In [ ]:
all_text = ' '.join(df['Question']) + ' ' + ' '.join(df['Answer']) + ' ' + ' '.join(df['Context'])
words = list(all_text.split())

len(words)

In [ ]:
from collections import Counter

word_counts = Counter(all_text.split())
word_counts

In [ ]:
unique_words = list(word_counts.keys())
unique_words

In [ ]:
print(len(unique_words))

In [ ]:
def compare_words_with_bart_vocab(word_list):
    # Tokenize the words in the list
    tokenized_words = [token for word in word_list for token in tokenizer.tokenize(word)]

    # Get BART's vocabulary
    bart_vocab = set(tokenizer.get_vocab().keys())

    # Check which tokens are in or not in BART's vocabulary
    words_in_vocab = {word for word in tokenized_words if word in bart_vocab}
    words_not_in_vocab = {word for word in tokenized_words if word not in bart_vocab}

    return words_in_vocab, words_not_in_vocab

In [ ]:
words_in_vocab, words_not_in_vocab = compare_words_with_bart_vocab(unique_words)

print("Words in BART vocabulary:", words_in_vocab)
print("Words not in BART vocabulary:", words_not_in_vocab)

In [ ]:
print(f"Words to be added: {len(words_not_in_vocab)}")

In [ ]:
def add_new_tokens_to_vocab(new_tokens):
    num_added_tokens = tokenizer.add_tokens(new_tokens)
    model.resize_token_embeddings(len(tokenizer))
    vocab = tokenizer.get_vocab()
    print("Vocabulary size:", len(vocab))

new_tokens = list(words_not_in_vocab)
num_added_tokens = add_new_tokens_to_vocab(new_tokens)

In [9]:
class QADataset(Dataset):
    '''For Loading the Dataset for Question Answering'''
    def __init__(self, data, tokenizer, question_max_length=200, context_max_length=200, answer_max_length=80):
        self.data = data
        self.tokenizer = tokenizer
        self.question_max_length = question_max_length
        self.context_max_length = context_max_length
        self.answer_max_length = answer_max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Extract the question, object (context), and answer from the dataset
        question = self.data.iloc[idx]['question']
        object_name = self.data.iloc[idx]['object']
        answer = self.data.iloc[idx]['answer']
        context = self.data.iloc[idx]['Context']

        # Combine the object name (context) and question
        combined = 'Object :' + str(object_name) + ' Question: ' + str(question) + ' Context: ' + str(context)            # if object_name is the float

        # Tokenize the combined input (question + context)
        inputs = self.tokenizer(
            combined,
            max_length=self.context_max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        targets = self.tokenizer(
            answer,
            max_length=self.answer_max_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        input_ids = inputs.input_ids.squeeze()
        attention_mask = inputs.attention_mask.squeeze()
        target_ids = targets.input_ids.squeeze()

        return {

            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': target_ids
        }

In [10]:
model

BartForConditionalGeneration(
  (model): BartModel(
    (shared): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): BartScaledWordEmbedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartSdpaAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
    

In [11]:
train_dataset = QADataset(train_df, tokenizer=tokenizer)
val_dataset = QADataset(val_df, tokenizer=tokenizer)
test_dataset = QADataset(test_df, tokenizer=tokenizer)

### Code changed with freezing the encoder layers, gradient accumulation and low learning rate

In [13]:
class VQA_Trainer:
    '''Class for Trainer Setup to Train the BART Model for VQA'''

    def __init__(self, model, train_dataloader, eval_dataloader, device, config, gradient_accumulation_steps=4, learning_rate=1e-5):
        ''' Constructor '''
        self.model = model
        self.train_dataloader = train_dataloader
        self.eval_dataloader = eval_dataloader
        self.device = device
        self.tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
        self.scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

        # Freeze initial encoder layers to prevent overfitting
        for param in self.model.model.encoder.parameters():
            param.requires_grad = False

        # Optimizer with a lower learning rate
        self.optimizer = AdamW(filter(lambda p: p.requires_grad, self.model.parameters()), lr=learning_rate)

        # Learning rate scheduler (adjust if necessary)
        total_steps = config["epochs"] * (len(train_dataloader) // gradient_accumulation_steps)
        self.scheduler = get_linear_schedule_with_warmup(self.optimizer, num_warmup_steps=0, num_training_steps=total_steps)

        # WandB setup
        wandb.init(project=config['project_name'], config=config)
        wandb.watch(self.model, log="all")

        self.gradient_accumulation_steps = gradient_accumulation_steps

    def evaluate(self):
        ''' For Evaluation at the End of Each Epoch '''
        self.model.eval()
        total_loss = 0
        predictions, references, token_level_accuracies = [], [], []

        progress_bar = tqdm(self.eval_dataloader, desc="Evaluating")
        for batch in progress_bar:
            with torch.no_grad():
                inputs = {key: val.to(self.device) for key, val in batch.items()}
                outputs = self.model(**inputs)
                total_loss += outputs.loss.item()

                # Generate predictions and decode
                summary_ids = self.model.generate(inputs['input_ids'], min_length=10, max_length=100, num_beams=4, early_stopping=True)
                decoded_preds = self.tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

                labels = batch['labels']
                labels = torch.where(labels != -100, labels, self.tokenizer.pad_token_id)
                decoded_refs = self.tokenizer.batch_decode(labels, skip_special_tokens=True)

                predictions.extend([pred.split() for pred in decoded_preds])
                references.extend([[ref.split()] for ref in decoded_refs])

                # Token-level accuracy
                for pred, ref in zip(decoded_preds, decoded_refs):
                    pred_tokens, ref_tokens = pred.split(), ref.split()
                    token_accuracy = sum(1 for p, r in zip(pred_tokens, ref_tokens) if p == r) / max(len(ref_tokens), 1)
                    token_level_accuracies.append(token_accuracy)

        avg_loss = total_loss / len(self.eval_dataloader)
        perplexity = math.exp(avg_loss)
        bleu_score = corpus_bleu(references, predictions)

        # ROUGE-L scores
        rouge_l_f1_scores, rouge_l_precision_scores, rouge_l_recall_scores = [], [], []
        for pred, ref in zip(decoded_preds, decoded_refs):
            rouge_l = self.scorer.score(ref, pred)['rougeL']
            rouge_l_f1_scores.append(rouge_l.fmeasure)
            rouge_l_precision_scores.append(rouge_l.precision)
            rouge_l_recall_scores.append(rouge_l.recall)

        avg_token_level_accuracy = np.mean(token_level_accuracies)
        avg_rouge_l_f1 = np.mean(rouge_l_f1_scores)
        avg_rouge_l_precision = np.mean(rouge_l_precision_scores)
        avg_rouge_l_recall = np.mean(rouge_l_recall_scores)

        # Log metrics
        metrics = {
            "Validation Loss": avg_loss,
            "Perplexity": perplexity,
            "BLEU": bleu_score,
            "ROUGE-L F1": avg_rouge_l_f1,
            "ROUGE-L Precision": avg_rouge_l_precision,
            "ROUGE-L Recall": avg_rouge_l_recall,
            "Token-Level Accuracy": avg_token_level_accuracy,
        }
        wandb.log(metrics)
        print(tabulate(pd.DataFrame([metrics]), headers="keys", tablefmt="psql"))

        return metrics

    def train_epoch(self):
        ''' To Train for Single Epoch with Gradient Accumulation '''
        self.model.train()
        total_loss = 0
        for i, batch in enumerate(tqdm(self.train_dataloader, desc="Training")):
            inputs = {key: val.to(self.device) for key, val in batch.items()}
            outputs = self.model(**inputs)
            loss = outputs.loss / self.gradient_accumulation_steps
            loss.backward()
            total_loss += loss.item()

            # Step only after gradient accumulation steps
            if (i + 1) % self.gradient_accumulation_steps == 0:
                self.optimizer.step()
                self.scheduler.step()
                self.optimizer.zero_grad()

            wandb.log({"train_loss": loss.item()})

        print(f"Total training loss for epoch: {total_loss / len(self.train_dataloader)}")

    def train(self, epochs):
        ''' To Train for N Number of Epochs Passed from User '''
        for epoch in range(epochs):
            print(f"Epoch {epoch + 1}/{epochs}")
            self.train_epoch()
            metrics = self.evaluate()
            print(f"Metrics: {metrics}")

        torch.save(self.model.state_dict(), f"model_final.pth")


In [14]:
api_key = os.getenv('API_KEY')
!wandb login --relogin $api_key

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
Traceback (most recent call last):
  File "/opt/conda/bin/wandb", line 8, in <module>
    sys.exit(cli())
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1688, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/conda/lib/python3.10/site-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/opt/conda/lib/python3.10/sit

In [15]:
config = {
     "batch_size":8,
     "epochs" : 10,
     "model_name": "facebook/bart-large-cnn",
     "project_name": "RAG_Conditional_BART",
}

In [16]:
train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=True)

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BartForConditionalGeneration.from_pretrained(config['model_name']).to(device)

In [18]:
trainer = VQA_Trainer(model, train_dataloader, val_dataloader,device,config)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wand

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [19]:
trainer.train(config['epochs'])

Epoch 1/10


Training: 100%|██████████| 594/594 [02:51<00:00,  3.47it/s]


Total training loss for epoch: 0.5514911087825644


Evaluating: 100%|██████████| 66/66 [01:43<00:00,  1.57s/it]


+----+-------------------+--------------+----------+--------------+---------------------+------------------+------------------------+
|    |   Validation Loss |   Perplexity |     BLEU |   ROUGE-L F1 |   ROUGE-L Precision |   ROUGE-L Recall |   Token-Level Accuracy |
|----+-------------------+--------------+----------+--------------+---------------------+------------------+------------------------|
|  0 |           1.03936 |      2.82742 | 0.150997 |     0.294921 |             0.38086 |         0.245384 |               0.115591 |
+----+-------------------+--------------+----------+--------------+---------------------+------------------+------------------------+
Metrics: {'Validation Loss': 1.039363952297153, 'Perplexity': 2.8274180695367317, 'BLEU': 0.15099734749378416, 'ROUGE-L F1': 0.29492116815589925, 'ROUGE-L Precision': 0.38086007130124777, 'ROUGE-L Recall': 0.24538371168305378, 'Token-Level Accuracy': 0.11559075947283412}
Epoch 2/10


...

+----+-------------------+--------------+----------+--------------+---------------------+------------------+------------------------+
|    |   Validation Loss |   Perplexity |     BLEU |   ROUGE-L F1 |   ROUGE-L Precision |   ROUGE-L Recall |   Token-Level Accuracy |
|----+-------------------+--------------+----------+--------------+---------------------+------------------+------------------------|
|  0 |          0.734266 |      2.08395 | 0.275009 |     0.616154 |             0.58632 |         0.683643 |                0.25073 |
+----+-------------------+--------------+----------+--------------+---------------------+------------------+------------------------+
Metrics: {'Validation Loss': 0.7342657103683009, 'Perplexity': 2.083951206789657, 'BLEU': 0.2750088220682335, 'ROUGE-L F1': 0.6161542862651733, 'ROUGE-L Precision': 0.5863204197763021, 'ROUGE-L Recall': 0.6836425957942986, 'Token-Level Accuracy': 0.2507301745016528}
Epoch 10/10


Training: 100%|██████████| 594/594 [02:58<00:00,  3.32it/s]


Total training loss for epoch: 0.1735944439867129


Evaluating: 100%|██████████| 66/66 [01:46<00:00,  1.61s/it]


+----+-------------------+--------------+----------+--------------+---------------------+------------------+------------------------+
|    |   Validation Loss |   Perplexity |     BLEU |   ROUGE-L F1 |   ROUGE-L Precision |   ROUGE-L Recall |   Token-Level Accuracy |
|----+-------------------+--------------+----------+--------------+---------------------+------------------+------------------------|
|  0 |          0.732179 |      2.07961 | 0.277023 |     0.394495 |            0.365138 |           0.4391 |               0.256147 |
+----+-------------------+--------------+----------+--------------+---------------------+------------------+------------------------+
Metrics: {'Validation Loss': 0.7321786979834238, 'Perplexity': 2.079606510106488, 'BLEU': 0.2770234858450497, 'ROUGE-L F1': 0.39449547742230673, 'ROUGE-L Precision': 0.3651382942000791, 'ROUGE-L Recall': 0.439100135975136, 'Token-Level Accuracy': 0.25614702294148145}


In [ ]:
wandb.finish()

In [ ]:
# free up unused GPU
import gc
import torch

gc.collect()
torch.cuda.empty_cache()